# Planet Simulator with Tensorflow Pipeline

Author: Craig Boger
06/07/2020

This script takes the model prototyping and learning of v1.01 and tries to expand upon it in 2 key areas.

1) Perform data normalization and processing in TF data libraries.
2) Take predictions in normalized form and output them in their unormalized form for use in simulation.

## Straight Up Just Stealing Someone's Code and Trying to Run It

Credit to benrules2: https://gist.github.com/benrules2/220d56ea6fe9a85a4d762128b11adfba

In [1]:
import math
import random
%matplotlib widget
import matplotlib.pyplot as plot
from mpl_toolkits.mplot3d import Axes3D

class point:
    def __init__(self, x,y,z):
        self.x = x
        self.y = y
        self.z = z

class body:
    def __init__(self, location, mass, velocity, name = ""):
        self.location = location
        self.mass = mass
        self.velocity = velocity
        self.name = name

def calculate_single_body_acceleration(bodies, body_index):
    G_const = 6.67408e-11 #m3 kg-1 s-2
    acceleration = point(0,0,0)
    target_body = bodies[body_index]
    for index, external_body in enumerate(bodies):
        if index != body_index:
            r = (target_body.location.x - external_body.location.x)**2 + (target_body.location.y - external_body.location.y)**2 + (target_body.location.z - external_body.location.z)**2
            r = math.sqrt(r)
            tmp = G_const * external_body.mass / r**3
            acceleration.x += tmp * (external_body.location.x - target_body.location.x)
            acceleration.y += tmp * (external_body.location.y - target_body.location.y)
            acceleration.z += tmp * (external_body.location.z - target_body.location.z)

    return acceleration

def compute_velocity(bodies, time_step = 1):
    for body_index, target_body in enumerate(bodies):
        acceleration = calculate_single_body_acceleration(bodies, body_index)

        target_body.velocity.x += acceleration.x * time_step
        target_body.velocity.y += acceleration.y * time_step
        target_body.velocity.z += acceleration.z * time_step 


def update_location(bodies, time_step = 1):
    for target_body in bodies:
        target_body.location.x += target_body.velocity.x * time_step
        target_body.location.y += target_body.velocity.y * time_step
        target_body.location.z += target_body.velocity.z * time_step

def compute_gravity_step(bodies, time_step = 1):
    compute_velocity(bodies, time_step = time_step)
    update_location(bodies, time_step = time_step)

def plot_output(bodies, outfile = None):
    fig = plot.figure()
    colours = ['r','b','g','y','m','c']
    ax = fig.add_subplot(1,1,1, projection='3d')
    max_range = 0
    for current_body in bodies: 
        max_dim = max(max(current_body["x"]),max(current_body["y"]),max(current_body["z"]))
        if max_dim > max_range:
            max_range = max_dim
        ax.plot(current_body["x"], current_body["y"], current_body["z"], c = random.choice(colours), label = current_body["name"])        
    
    ax.set_xlim([-max_range,max_range])    
    ax.set_ylim([-max_range,max_range])
    ax.set_zlim([-max_range,max_range])
    ax.legend()        

    if outfile:
        plot.savefig(outfile)
    else:
        plot.show()

def run_simulation(bodies, names = None, time_step = 1, number_of_steps = 10000, report_freq = 100):

    #create output container for each body
    body_locations_hist = []
    for current_body in bodies:
        body_locations_hist.append({"x":[], "y":[], "z":[], "name":current_body.name})
        
    for i in range(1,number_of_steps):
        compute_gravity_step(bodies, time_step = 1000)            
        
        if i % report_freq == 0:
            for index, body_location in enumerate(body_locations_hist):
                body_location["x"].append(bodies[index].location.x)
                body_location["y"].append(bodies[index].location.y)           
                body_location["z"].append(bodies[index].location.z)       

    return body_locations_hist        
            
#planet data (location (m), mass (kg), velocity (m/s)
sun = {"location":point(0,0,0), "mass":2e30, "velocity":point(0,0,0)}
mercury = {"location":point(0,5.7e10,0), "mass":3.285e23, "velocity":point(47000,0,0)}
venus = {"location":point(0,1.1e11,0), "mass":4.8e24, "velocity":point(35000,0,0)}
earth = {"location":point(0,1.5e11,0), "mass":6e24, "velocity":point(30000,0,0)}
mars = {"location":point(0,2.2e11,0), "mass":2.4e24, "velocity":point(24000,0,0)}
jupiter = {"location":point(0,7.7e11,0), "mass":1e28, "velocity":point(13000,0,0)}
saturn = {"location":point(0,1.4e12,0), "mass":5.7e26, "velocity":point(9000,0,0)}
uranus = {"location":point(0,2.8e12,0), "mass":8.7e25, "velocity":point(6835,0,0)}
neptune = {"location":point(0,4.5e12,0), "mass":1e26, "velocity":point(5477,0,0)}
pluto = {"location":point(0,3.7e12,0), "mass":1.3e22, "velocity":point(4748,0,0)}


if __name__ == "__main__":

    #build list of planets in the simulation, or create your own
    bodies = [
        body( location = sun["location"], mass = sun["mass"], velocity = sun["velocity"], name = "sun"),
        body( location = mercury["location"], mass = mercury["mass"], velocity = mercury["velocity"], name = "mercury"),
        body( location = venus["location"], mass = venus["mass"], velocity = venus["velocity"], name = "venus"),
        body( location = earth["location"], mass = earth["mass"], velocity = earth["velocity"], name = "earth"),
        body( location = mars["location"], mass = mars["mass"], velocity = mars["velocity"], name = "mars"),
        body( location = jupiter["location"], mass = jupiter["mass"], velocity = jupiter["velocity"], name = "jupiter"),
        body( location = saturn["location"], mass = saturn["mass"], velocity = saturn["velocity"], name = "saturn"),
        body( location = uranus["location"], mass = uranus["mass"], velocity = uranus["velocity"], name = "uranus"),
        body( location = neptune["location"], mass = neptune["mass"], velocity = neptune["velocity"], name = "neptune"),
        body( location = pluto["location"], mass = pluto["mass"], velocity = pluto["velocity"], name = "pluto")
        ]
    
    # Original defaults of simulation
    # motions = run_simulation(bodies, time_step = 100, number_of_steps = 80000, report_freq = 1000)
    # Try messing with report frequency to get more data.
    motions = run_simulation(bodies, time_step = 10, number_of_steps = 6000000, report_freq = 100)
    plot_output(motions, outfile = 'orbits.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Take motions data from the above simulation and convert it to a Pandas dataframe.  The "motions" output is a list of python dictionaries that can be converted into a dataframe and then manipulated.

In [2]:
import pandas as pd
import numpy as np

motions_df = pd.DataFrame(motions)
motions_df.head(100)

,x,y,z,name
0,"[6.17261536148749, 49.37902931004262, 166.6238...","[6062.379510449177, 24129.08432936523, 54198.9...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",sun
1,"[4694355206.032213, 9354859805.615133, 1394777...","[56792631341.03519, 56175843290.39122, 5515329...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",mercury
2,"[3499415066.4596167, 6995320910.098446, 104842...","[109944304443.26683, 109778376891.15071, 10950...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",venus
3,"[2999802268.7561436, 5998418130.146453, 899466...","[149970049806.4969, 149880804295.7356, 1497322...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",earth
4,"[2399949856.0031266, 4799598821.678605, 719864...","[219986083536.2445, 219944611164.44067, 219875...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",mars
5,"[1299999366.479779, 2599994931.4595885, 389998...","[769998863558.2162, 769995476739.6332, 7699898...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",jupiter
6,"[899999928.8199571, 1799999430.5168397, 269999...","[1399999647604.3037, 1399998597395.4744, 13999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",saturn
7,"[683499993.1607178, 1366999945.2816358, 205049...","[2799999913115.264, 2799999654181.5522, 279999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",uranus
8,"[547699998.6801858, 1095399989.4406931, 164309...","[4499999966439.355, 4499999866422.006, 4499999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",neptune
9,"[474799997.95801955, 949599983.6629324, 142439...","[3699999950348.1753, 3699999802375.9053, 36999...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",pluto


In [3]:
# Trying to separate out each row of list or dataframe into its own dataframe.
# Will later put these dataframes back together into 1 large dataframe.

motions_df_list = []
for body in motions:
    motions_df_list.append(pd.DataFrame(body))

In [4]:
motions_df_list[3]

,x,y,z,name
0,2.999802e+09,1.499700e+11,0.0,earth
1,5.998418e+09,1.498808e+11,0.0,earth
2,8.994662e+09,1.497323e+11,0.0,earth
3,1.198735e+10,1.495246e+11,0.0,earth
4,1.497529e+10,1.492578e+11,0.0,earth
...,...,...,...,...
59994,2.689581e+11,-6.753826e+10,0.0,earth
59995,2.676697e+11,-6.488198e+10,0.0,earth
59996,2.664329e+11,-6.220061e+10,0.0,earth
59997,2.652481e+11,-5.949513e+10,0.0,earth


In [5]:
# Combine the dataframes into a single, large dataframe.
# Can later choose a planet to be the target we train to predict.
complete_motion_df = None

for body in motions_df_list:
    # Append name of body to each column and remove the name column
    body_name = body.loc[0, "name"]
    body.columns = [body_name + "_x",
                    body_name + "_y",
                    body_name + "_z",
                    "name"]
    # Add current body to the complete dataframe.
    complete_motion_df = pd.concat([complete_motion_df, body.iloc[:, 0:3]], axis=1)

complete_motion_df.head(100)

,sun_x,sun_y,sun_z,mercury_x,mercury_y,mercury_z,venus_x,venus_y,venus_z,earth_x,...,saturn_z,uranus_x,uranus_y,uranus_z,neptune_x,neptune_y,neptune_z,pluto_x,pluto_y,pluto_z
0,6.172615e+00,6.062380e+03,0.0,4.694355e+09,5.679263e+10,0.0,3.499415e+09,1.099443e+11,0.0,2.999802e+09,...,0.0,6.835000e+08,2.800000e+12,0.0,5.477000e+08,4.500000e+12,0.0,4.748000e+08,3.700000e+12,0.0
1,4.937903e+01,2.412908e+04,0.0,9.354860e+09,5.617584e+10,0.0,6.995321e+09,1.097784e+11,0.0,5.998418e+09,...,0.0,1.367000e+09,2.800000e+12,0.0,1.095400e+09,4.500000e+12,0.0,9.496000e+08,3.700000e+12,0.0
2,1.666238e+02,5.419895e+04,0.0,1.394778e+10,5.515330e+10,0.0,1.048421e+10,1.095024e+11,0.0,8.994662e+09,...,0.0,2.050500e+09,2.799999e+12,0.0,1.643100e+09,4.500000e+12,0.0,1.424400e+09,3.700000e+12,0.0
3,3.948516e+02,9.627002e+04,0.0,1.843961e+10,5.373113e+10,0.0,1.396259e+10,1.091166e+11,0.0,1.198735e+10,...,0.0,2.734000e+09,2.799999e+12,0.0,2.190800e+09,4.499999e+12,0.0,1.899200e+09,3.699999e+12,0.0
4,7.709150e+02,1.503396e+05,0.0,2.279721e+10,5.191794e+10,0.0,1.742697e+10,1.086215e+11,0.0,1.497529e+10,...,0.0,3.417499e+09,2.799998e+12,0.0,2.738500e+09,4.499999e+12,0.0,2.374000e+09,3.699999e+12,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4.138930e+06,5.399689e+07,0.0,3.050288e+10,-4.133027e+10,0.0,1.578892e+10,-1.109137e+11,0.0,1.441446e+11,...,0.0,6.560995e+10,2.799207e+12,0.0,5.257803e+10,4.499694e+12,0.0,4.557899e+10,3.699547e+12,0.0
96,4.259370e+06,5.511102e+07,0.0,2.606635e+10,-4.407848e+10,0.0,1.238219e+10,-1.113485e+11,0.0,1.431961e+11,...,0.0,6.629326e+10,2.799191e+12,0.0,5.312570e+10,4.499687e+12,0.0,4.605374e+10,3.699537e+12,0.0
97,4.381922e+06,5.623611e+07,0.0,2.137176e+10,-4.638955e+10,0.0,8.963734e+09,-1.116778e+11,0.0,1.421936e+11,...,0.0,6.697656e+10,2.799174e+12,0.0,5.367336e+10,4.499681e+12,0.0,4.652848e+10,3.699528e+12,0.0
98,4.506591e+06,5.737214e+07,0.0,1.646399e+10,-4.823708e+10,0.0,5.536787e+09,-1.119012e+11,0.0,1.411373e+11,...,0.0,6.765986e+10,2.799157e+12,0.0,5.422102e+10,4.499674e+12,0.0,4.700322e+10,3.699518e+12,0.0


In [6]:
complete_motion_df.shape

(59999, 30)

In [7]:
# Save the simulation data for later loading and use
complete_motion_df.to_csv("raw_model_output.csv", index=False)

At this point, we have a single dataframe with all bodies and all positions with each time step as the index of our rows.

# Trying to Create a tf.data Dataset from the Constructed, Unrandomized, Unnormalized Data

### Imports

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
# Probably not needed since not using regressor or doing any feature engineering.
import sklearn
from sklearn.preprocessing import StandardScaler  # Scaler for normalizing data.
from sklearn.preprocessing import MinMaxScaler  # Scaler for normalizing data.
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"
# Recommended to enable eager execution when developing model.
# Processing data: https://www.youtube.com/watch?v=oFFbKogYdfc
# tf.enable_eager_execution()

# Import Keras
from tensorflow import keras

# to make this notebook's output stable across runs
#np.random.seed(42)

# Use sklearn for data processing


# Common imports
import numpy as np
import os

In [ ]:
tf.__version__

In [ ]:
keras.__version__

## Start Here with Trying to Process Data with Tensorflow Datasets

One of the difficulties is using a mixture of numpy, pandas, and sklearn to take input data (influx), arrange it, split it out, normalize it, and then train a model.  With tf.data input pipelines (similar to sklearn pipelines), we can create data and machine learning pipelines for training or inference.  This allows us to encapsulate not only the machine learning into a Tensorflow model, but the necessary transformations to that data.  That allows us to deploy the model and data transformations as a single object to the later simulator. \
The input pipeline let's us take raw data from any source, like csv, numpy arrays, distributed file system, etc, and convert it into the tensors we will use to train our model.

Intro to tensors:
https://www.tensorflow.org/guide/tensor

Good source on how data loading and preprocessing is usually done: https://stackoverflow.com/questions/55321905/want-to-split-train-and-test-data-gotten-from-a-csv-with-tensorflow
1) Load the data into memory with numpy
2) Split the data into train and validation

Since we are not using a massive dataset, then we might be able to use tf.split to split an exsting tf dataset into train and validation.
https://docs.w3cub.com/tensorflow~python/tf/split/

### Creating Numpy Version of the Data as a Backup

Create copy of the complete dataframe and shuffle it using pandas.

In [ ]:
copy1_complete_motion_df = complete_motion_df.copy()
copy1_complete_motion_df = copy1_complete_motion_df.sample(frac=1).reset_index(drop=True)
copy1_complete_motion_df.head(10)

Split out the target x,y,z columns as the last 3 columns in the dataframe.  Skipping scaling and normalization.

In [ ]:
# Assuming last 3 columns in the dataframe are the target x,y, and z values.  
target = copy1_complete_motion_df.iloc[:,-3:]
# Drop target from main dataframe.
copy1_complete_motion_df.drop(copy1_complete_motion_df.iloc[:,-3:], axis = 1, inplace = True)
target.head(5)

Split the x, y, and z coordinates out for the target to use a specific dataset for each possible coordinate output.

In [ ]:
target_x = target.iloc[:,0]
target_y = target.iloc[:,1]
target_z = target.iloc[:,2]

Convert all pandas dataframes to numpy arrays so they are compatible with Tensorflow.

In [ ]:
complete_motion_np = copy1_complete_motion_df.to_numpy()
target_np = target.to_numpy()
target_x_np = target_x.to_numpy()
target_y_np = target_y.to_numpy()
target_z_np = target_z.to_numpy()

Split into train, validation, and test datasets.

In [ ]:
#Split into train, validation, and test sets.
# Setup train, validation, and test splits
DATASET_SIZE = len(complete_motion_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

X_train, X_valid, X_test = complete_motion_np[:train_size], complete_motion_np[train_size:(train_size+val_size)], complete_motion_np[(train_size + val_size):]
y_train_x, y_valid_x, y_test_x = target_x_np[:train_size], target_x_np[train_size:(train_size+val_size)], target_x_np[(train_size + val_size):]
y_train_y, y_valid_y, y_test_y = target_y_np[:train_size], target_y_np[train_size:(train_size+val_size)], target_y_np[(train_size + val_size):]
y_train_z, y_valid_z, y_test_z = target_z_np[:train_size], target_z_np[train_size:(train_size+val_size)], target_z_np[(train_size + val_size):]

### Creating a tf dataset from slices (numpy array, pandas dataframe, etc). 
https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

Probably one of the better articles on using tensorflow datasets: \
https://adventuresinmachinelearning.com/tensorflow-dataset-tutorial/

TF documentation on tf.data: Building Tensorflow Input Pipelines: \
https://www.tensorflow.org/guide/data

Method for splitting tensorflow datasets into train, validation, and test: \
https://stackoverflow.com/questions/51125266/how-do-i-split-tensorflow-datasets/51126863

In [ ]:
#Split the dataset into input and targets for the x, y, and z coordinates.
# Assuming last 3 columns in the dataframe are the target x,y, and z values.  
target = complete_motion_df.iloc[:,-3:]
# Drop target from main dataframe.
complete_motion_df.drop(complete_motion_df.iloc[:,-3:], axis = 1, inplace = True)
# Split the x, y, and z coordinates out for the target to use a specific dataset for each possible coordinate output.
# Convert targets to numpy arrays as well so we can use them in the model.
target_x_np = target.iloc[:,0].to_numpy()
target_y_np = target.iloc[:,1].to_numpy()
target_z_np = target.iloc[:,2].to_numpy()
# Usually training, validation, and test data would be coming from different CSV files or sources.
# complete_motion_df only consists of input data at this point.
complete_motion_np = complete_motion_df.to_numpy()

#Create one large tensorflow dataset.
full_dataset = tf.data.Dataset.from_tensor_slices((complete_motion_np, 
                                                   target_x_np, 
                                                   target_y_np,
                                                   target_z_np)
                                                 )

In [ ]:
complete_motion_np[0].shape

In [ ]:
full_dataset.element_spec

In [ ]:
# Iterate through dataset and print the input and targets.
# Will select top 5 to iterate through.
for feat, targ_x, targ_y, targ_z in full_dataset.take(5):
    print('Features: {} Target_X: {} Target_Y: {} Target_Z: {}'.format(feat, targ_x, targ_y, targ_z))

In [ ]:
# Shuffle the full dataset before splitting into train, validation, and test.
# Since dataset can fit in memory, can set buffer to be the size of the data.
full_dataset_num_samples = complete_motion_df.shape[0]  #Get the size of the dataset to set the randomize buffer
#full_dataset = full_dataset.shuffle(buffer_size=full_dataset_num_samples).batch(1)
full_dataset = full_dataset.shuffle(buffer_size=full_dataset_num_samples)

In [ ]:
full_dataset.element_spec

In [ ]:
#Split into train, validation, and test sets.
# Setup train, validation, and test splits
DATASET_SIZE = len(complete_motion_df)
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)
# Take the shuffled dataset and split into train, validation, and test datasets.
train_dataset = full_dataset.take(train_size)   # Take top of dataset for training data
test_dataset = full_dataset.skip(train_size)    # Take the rest of the dataset for validation and test
val_dataset = test_dataset.skip(test_size)      # Take a part of the test data for validation during training
test_dataset = test_dataset.take(test_size)     # Get rid of the validation data from the test dataset

# Try a Quick Neural Net for Predicting Jupiter's Position

Notes: \
<br>
Instead of using sklearn to normalize or manually making a normalization and standardization layer like p. 431 of the book, try using at least 1 Batch normalization layer after the input layer.  Can also add after hidden layers. \
<br>
Might need to add an activation function to the output layer later to help with scaling of the data.


## Try Creating Single Input, Multiple Output Regression Model

Trying to create a regression NN where instead of designating an output layer of 3 nodes, 3 output layers of a single node are used to designate specific datasets and loss functions.  Still need to figure out later how to get a 3 node output to correspond to the input training data.

Use functional API to build basic NN architecture.

In [ ]:
# Functions with different versions of the neural network.

def get_model1(input_shape):
    # Use functional API to build basic NN architecture.
    input_main = keras.layers.Input(shape=input_shape)
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal")(normal1)
    normal2 = keras.layers.BatchNormalization()(hidden1)
    hidden2 = keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal")(normal2)
    normal3 = keras.layers.BatchNormalization()(hidden2)
    output_x = keras.layers.Dense(1, activation="linear", name="output_x")(normal3)
    output_y = keras.layers.Dense(1, activation="linear", name="output_y")(normal3)
    output_z = keras.layers.Dense(1, activation="linear", name="output_z")(normal3)
    # Best parameters so far: keras.optimizers.RMSprop(lr=0.1, rho=0.9)
    return keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

def get_model_2(input_shape):
    # Use functional API to build basic NN architecture.
    input_main = keras.layers.Input(shape=input_shape)
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(300, activation="tanh")(normal1)
    normal2 = keras.layers.BatchNormalization()(hidden1)
    hidden2 = keras.layers.Dense(100, activation="tanh")(normal2)
    normal3 = keras.layers.BatchNormalization()(hidden2)
    output_x = keras.layers.Dense(1, name="output_x")(normal3)
    output_y = keras.layers.Dense(1, name="output_y")(normal3)
    output_z = keras.layers.Dense(1, name="output_z")(normal3)
    # Best parameters so far: keras.optimizers.RMSprop(lr=0.1, rho=0.9)
    return keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

def get_model_3(input_shape):
    input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(1000, activation="elu", kernel_initializer="he_normal")(normal1)
    normal2 = keras.layers.BatchNormalization()(hidden1)
    hidden2 = keras.layers.Dense(1000, activation="elu", kernel_initializer="he_normal")(normal2)
    normal3 = keras.layers.BatchNormalization()(hidden2)
    output_x = keras.layers.Dense(1, name="output_x")(normal3)
    output_y = keras.layers.Dense(1, name="output_y")(normal3)
    output_z = keras.layers.Dense(1, name="output_z")(normal3)

    model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])
    
    input_optimizer = keras.optimizers.RMSprop(lr=10, rho=0.9)
    
def get_model_4(input_shape):
    input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
    normal1 = keras.layers.BatchNormalization()(input_main)
    hidden1 = keras.layers.Dense(1000, activation="selu", kernel_initializer="lecun_normal")(normal1)
    hidden2 = keras.layers.Dense(1000, activation="selu", kernel_initializer="lecun_normal")(hidden1)
    output_x = keras.layers.Dense(1, name="output_x")(hidden2)
    output_y = keras.layers.Dense(1, name="output_y")(hidden2)
    output_z = keras.layers.Dense(1, name="output_z")(hidden2)

    model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])
    input_optimizer = keras.optimizers.RMSprop(lr=2, rho=0.9)
    
def get_model_5(input_shape):  #Best one yet.  Typically takes about 1500 epochs to get decend results.
    # Use functional API to build basic NN architecture.
    input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
    hidden1 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(input_main)
    hidden2 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(hidden1)
    output_x = keras.layers.Dense(1, activation="linear", name="output_x")(hidden2)
    output_y = keras.layers.Dense(1, activation="linear", name="output_y")(hidden2)
    output_z = keras.layers.Dense(1, activation="linear", name="output_z")(hidden2)

    model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

    #Set 
    input_losses = ["mae", "mae", "mae"]
    input_loss_weights = [0.4, 0.4, 0.2]
    input_optimizer = keras.optimizers.RMSprop(lr=0.0000005, rho=0.01)
    input_metrics = ["mae"]
    input_num_epochs = 200
    input_batch_size = 64

    model.summary()
    #Also can use and probably should use Adam
    input_optimizer = keras.optimizers.Adam(learning_rate=1e-5)


Create model with specified input and output layers

In [ ]:
# Create model with specified input and output layers.
# Select which model to try.
# Pass shape of input layer to the function.
#model = get_model_2(complete_motion_np.shape[1:])

# Use functional API to build basic NN architecture.
input_main = keras.layers.Input(shape=complete_motion_np.shape[1:])
hidden1 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(input_main)
hidden2 = keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")(hidden1)
output_x = keras.layers.Dense(1, activation="linear", name="output_x")(hidden2)
output_y = keras.layers.Dense(1, activation="linear", name="output_y")(hidden2)
output_z = keras.layers.Dense(1, activation="linear", name="output_z")(hidden2)

model =  keras.Model(inputs=[input_main], outputs=[output_x, output_y, output_z])

#Set 
input_losses = ["msle", "msle", "msle"]
input_loss_weights = [0.4, 0.4, 0.2]
input_optimizer = keras.optimizers.Adam(learning_rate=1e-7, beta_1=0.9, beta_2=0.999)
input_metrics = ["msle"]
input_num_epochs = 500
input_batch_size = 128

model.summary()

In [ ]:
#Before fitting the model, create callbacks for the various stages.

# Callback to implement overfitting.  Helps with regularization.  
# Keep from over-training.  Stops training when validation error starts increasing again.
# https://lambdalabs.com/blog/tensorflow-2-0-tutorial-04-early-stopping/
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='loss',
                                                 min_delta=0.0001,
                                                 patience=20)

# Callback for learning rate scheduling.  This way we can start with a higher learning rate then reduce as we go.
# Reducing the learning rate by a factor of "factor" every so many epochs or "patience".
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=50)

#Create list of all callbacks.
#callback_list = [early_stopping_cb, lr_scheduler]
callback_list = [lr_scheduler]

In [ ]:
# Compile model with specified loss functions for each output and specify weighting to provide each output.
# Weighting X and Y output more than Z
model.compile(loss=input_losses, 
              loss_weights=input_loss_weights, 
              optimizer=input_optimizer,
              metrics=input_metrics)

Train the model with separate x, y, z training sets.  Choose either numpy data or tensorflow dataset.

In [ ]:
# Fit the model using numpy formatted training and validation data.
history = model.fit(
    [X_train], [y_train_x, y_train_y, y_train_z],
    epochs=input_num_epochs,
    validation_data=([X_valid], [y_valid_x, y_valid_y, y_valid_z]),
    batch_size=input_batch_size,
    callbacks=callback_list
)

In [ ]:
# Convert training history to dataframe for analysis and plotting.
complete_history_data = pd.DataFrame(history.history)
complete_history_data.head(-9)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
complete_history_data[["output_x_msle", "val_output_x_msle"]].plot()

In [ ]:
# Create figure of subplots to plot total loss, x coordinate loss, y coordinate loss, and z coordinate MSEs.
fig2, mse_plots = plt.subplots(2,2)


#plot losses in each quadrant of the figure.
mse_plots[0][0].plot(complete_history_data[["output_x_msle", "val_output_x_msle"]])
#mse_plots[0][0].set_ylim(0,1)

mse_plots[0][1].plot(complete_history_data[["output_y_msle", "val_output_y_msle"]])
#mse_plots[0][1].set_ylim(0,1)

mse_plots[1][0].plot(complete_history_data[["output_z_msle", "val_output_z_msle"]])
#mse_plots[1][0].set_ylim(0,1)

plt.show()


In [ ]:
# Create figure of subplots to plot total loss, x coordinate loss, y coordinate loss, and z coordinate loss.
fig, loss_plots = plt.subplots(2,2)


#plot losses in each quadrant of the figure.
loss_plots[0][0].plot(complete_history_data[["loss", "val_loss"]])
#loss_plots[0][0].set_ylim(0,1)

loss_plots[0][1].plot(complete_history_data[["output_x_loss", "val_output_x_loss"]])
#loss_plots[0][1].set_ylim(0,1)

loss_plots[1][0].plot(complete_history_data[["output_y_loss", "val_output_y_loss"]])
#loss_plots[1][0].set_ylim(0,1)

loss_plots[1][1].plot(complete_history_data[["output_z_loss", "val_output_z_loss"]])
#loss_plots[1][1].set_ylim(0,1)


plt.show()


### Evaluate the Model with Test Data

In [ ]:
y_test_x.shape

In [ ]:
model.evaluate([X_test],[y_test_x, y_test_y, y_test_z])

### Predict Values and Inspect Differences

In [ ]:
x_pred, y_pred, z_pred = model.predict([X_test])

In [ ]:
pred_model_comparison = pd.DataFrame(data=np.concatenate((x_pred, y_test_x.reshape(-1,1), y_pred, y_test_y.reshape(-1,1), z_pred, y_test_z.reshape(-1,1)), axis=1),
                                    columns=['pred_x', 'model_x', 'pred_y', 'model_y', 'pred_z', 'model_z'])
pred_model_comparison.head(10)

In [ ]:
pred_model_comparison[["pred_x", "model_x"]].plot()

In [ ]:
pred_model_comparison[["pred_y", "model_y"]].plot()

In [ ]:
pred_model_comparison[["pred_z", "model_z"]].plot()